In [1]:
%reload_ext autoreload
%autoreload 2

import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [2]:
N_FOLD = 10
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = '501-1dCNN'
DATA_NAME = '501'
mkdir(f'./models/{SOL_NAME}/')

In [3]:
# CONSTANT
MEAN = -5.762330803300896
STD = 0.6339307835941186
EPS = 1e-9

In [4]:
# get ids
list_stock_id = get_stock_id()
list_time_id = get_time_id()

# Functions

In [5]:
def transform_target(target):
    return (np.log(target + EPS) - MEAN) / STD

def inverse_target(target):
    return np.exp(MEAN + STD * target) - EPS

def np_rmspe(y_true, y_pred):
    y_true = inverse_target(y_true)
    y_pred = inverse_target(y_pred)
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def mspe_loss(y_true, y_pred):
    y_true = K.exp(MEAN + STD * y_true) - EPS
    y_pred = K.exp(MEAN + STD * y_pred) - EPS
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))

def rmspe_keras(y_true, y_pred):
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))

In [15]:
def create_1dcnn(num_columns, num_labels, learning_rate):
    # input
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    # 1dcnn
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Reshape((16, 16))(x)
    x = tf.keras.layers.Conv1D(filters=12,
                      kernel_size=2,
                      strides=1,
                      activation='swish')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    
    # ffn
    for i in range(3):
        x = tf.keras.layers.Dense(64 // (2 ** i), activation='swish')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.GaussianNoise(0.01)(x)
        x = tf.keras.layers.Dropout(0.20)(x)
    x = tf.keras.layers.Dense(num_labels)(x)
    
    model = tf.keras.models.Model(inputs=inp, outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=mspe_loss,
    )
    return model

# Loading data

In [7]:
# train
df_train = dt.fread(f'./dataset/train_{DATA_NAME}_NN.csv').to_pandas()
fea_cols = [f for f in df_train.columns if f.startswith('B_') or f.startswith('T_') or f.startswith('Z_')]
# result
df_result = dt.fread('./dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [8]:
fea_cols_TA = [f for f in fea_cols if 'min_' not in f]
df_time_mean = df_train.groupby('time_id')[fea_cols_TA].mean()
df_time_mean.columns = [f'{c}_TA_mean' for c in df_time_mean.columns]
df_time_mean = df_time_mean.reset_index()
df_train = df_train.merge(df_time_mean, on='time_id', how='left')

del df_time_mean
gc.collect()

43

In [9]:
df_train['target'] = transform_target(df_train['target'])
df_train = gen_row_id(df_train)
df_train = add_time_fold(df_train, N_FOLD)

# Evaluation

In [10]:
def add_time_stats(df_train):
    time_cols = [f for f in df_train.columns if f.endswith('_time')]
    df_gp_stock = df_train.groupby('stock_id')
    #
    df_stats = df_gp_stock[time_cols].mean().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_mean' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].std().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_std' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].skew().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_skew' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].min().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_min' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].max().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_max' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].quantile(0.25).reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_q1' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].quantile(0.50).reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_q2' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].quantile(0.75).reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_q3' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    return df_train

In [11]:
batch_size = 1024
learning_rate = 6e-3
epochs = 1000

list_seeds = [0, 11, 42, 777, 2045]

In [17]:
list_rmspe = []
for i_seed, seed in enumerate(list_seeds):
    df_train = add_time_fold(df_train, N_FOLD, seed=seed)
    list_rmspe += [[]]
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]
        df_tr = add_time_stats(df_tr)
        df_te = add_time_stats(df_te)

        fea_cols = [f for f in df_tr if f.startswith('B_') or f.startswith('T_') or f.startswith('Z_')]

        X_train = df_tr[fea_cols].values
        y_train = df_tr[['target']].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['target']].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_seed+1}/{len(list_seeds)} | {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)


        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_{i_seed}_{i_fold}.hdf5'
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_delta=1e-5, verbose=2)
        es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=31, restore_best_weights=True, verbose=2)

        model = create_1dcnn(X_train.shape[1], 1, learning_rate)
        history = model.fit(X_train, y_train,
            epochs=epochs,
            validation_data=(X_test, y_test),
            validation_batch_size=len(y_test),
            batch_size=batch_size,
            verbose=2,
            callbacks=[rlr, es]
        ) 
        # model = tf.keras.models.load_model(ckp_path, custom_objects={'mspe_loss': mspe_loss})

        y_pred = model.predict(X_test, batch_size=len(y_test))
        curr_rmspe = np_rmspe(y_test, y_pred)
        list_rmspe[-1] += [curr_rmspe]
        model.save(ckp_path)
        # generate and save preds
        df_result.loc[idx_test, f'pred_{i_seed}'] = inverse_target(y_pred)
        clear_output()
        print(list_rmspe)

[[0.21022636450052495, 0.21939386684082246, 0.20975052519321782, 0.2085773537791255, 0.2113484591068515, 0.2119121523184086, 0.21110068228865625, 0.21208264595637888, 0.21416047509182187, 0.22773273338705263], [0.22096742507383604, 0.20646272827577072, 0.2115436938575406, 0.21968584354115303, 0.22984816197720548, 0.21101167408065497, 0.21269305781660341, 0.21244591125991594, 0.20925710000483455, 0.2111199499437056], [0.2131945100539358, 0.2154680645073522, 0.21478817587600602, 0.21147221223707088, 0.20577207180073784, 0.22450232155237212, 0.22015399685845816, 0.2091533701698531, 0.2094901784494245, 0.21914447248675742], [0.21141307303191084, 0.20700794105401787, 0.21007405037706894, 0.2241200798910265, 0.21281092406777036, 0.2131235356880075, 0.21761252555658234, 0.20780915023404992, 0.2187906140839999, 0.22313538070705846], [0.21206174502296368, 0.20933323728227768, 0.21184183928517433, 0.21033346929252794, 0.21737091357149843, 0.21735295651157147, 0.21245263600520106, 0.2179923217900

In [18]:
df_result.to_csv(f'./results/{SOL_NAME}.csv', index=False)

In [19]:
for i in range(len(list_seeds)):
    print(i, rmspe(df_result['target'], df_result[f'pred_{i}']))
print('All: ', rmspe(df_result['target'], df_result[[f'pred_{i}' for i in range(len(list_seeds))]].mean(axis=1)))

0 0.21369911137011632
1 0.21460543478949706
2 0.21438267580899686
3 0.2146666805788994
4 0.21427824604107953
All:  0.21010537596822407
